In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
# %load analise_debito.py

from datetime import datetime, timedelta
from pyspark.sql import Window
from pyspark.sql.types import StructType, IntegerType, StructField, StringType, BooleanType, TimestampType, DoubleType
from pyspark.sql.functions import col, rowNumber, format_string, length, concat, substring, lit, when, to_date
from pyspark.sql.functions import datediff, sum, count, pow, first, lag, desc, log10, regexp_extract

# Conexão fora do pyspark
# from pyspark import SparkContext
# from pyspark.sql import HiveContext
# sc = SparkContext('local[*]')
# sqlContext = HiveContext(sc)

# Função auxiliar de filtragem de dados mais recentes, segundo timestamp de atualização
def latest_contcur(df, ids):
    return df \
        .withColumn('_row', rowNumber().over(
            Window \
                .partitionBy(*[col(id) for id in ids]) \
                .orderBy(col('contcur').desc()))) \
        .filter(col('_row') == 1) \
        .drop('_row')

# Tabelas de origem no Hive
mpdt004_df = sqlContext.table('mp.mpdt004')
mpdt014_df = sqlContext.table('mp.mpdt014')
mpdt090_df = sqlContext.table('mp.mpdt090')
base_contestacao_df = sqlContext.table('arq_fraude.base_contestacao')

# Obter data de carga mais recente
dat_ref_carga = sqlContext.sql('select max(dat_ref_carga) from mp.mpdt014').collect()[0][0]
print('dat_ref_carga = ' + dat_ref_carga)

# Condição de filtro de última carga
ultima_carga = col('dat_ref_carga') == dat_ref_carga

mpdt004_latest_df = latest_contcur(mpdt004_df, ['siaidcd'])

mpdt090_latest_df = latest_contcur(mpdt090_df, ['codent', 'centalta', 'cuenta', 'impdiv', 'fecfac', 'siaidcd']) # + ['codcom']

# Contratos primários de cartões
#  & mpdt014_df.fecbaja.isNotNull()
contratos_df = latest_contcur(mpdt014_df.filter(ultima_carga), ['codent', 'centalta', 'cuenta', 'ctacargo']) \
    .filter((mpdt014_df.indcta == 'P')) \
    .select(mpdt014_df.codent, mpdt014_df.centalta, mpdt014_df.cuenta, mpdt014_df.ctacargo) \
    .distinct()

In [ ]:
contratos_df.count()

In [ ]:
transactions_df = mpdt090_latest_df \
    .filter(mpdt090_latest_df.codcom != '') \
    .join(
    contratos_df,
    (mpdt090_latest_df.codent == contratos_df.codent) &
    (mpdt090_latest_df.centalta == contratos_df.centalta) &
    (mpdt090_latest_df.cuenta == contratos_df.cuenta), 'inner') \
    .drop(contratos_df.codent) \
    .drop(contratos_df.centalta) \
    .drop(contratos_df.cuenta)

In [ ]:
transactions_df.count()

In [ ]:
# Base de compras contestadas
blah_df = base_contestacao_df \
    .filter(
        ultima_carga &
        (base_contestacao_df.descricaotransacao == 'Compras') &
        (base_contestacao_df.canal == 'AUTO ATENDIMENTO')) \
    .withColumn('numagencia_fix', regexp_extract(base_contestacao_df.numagencia, '(\d+)', 1))

compras_contestadas_df = blah_df \
    .filter(length(blah_df.numagencia_fix) > 0) \
    .withColumn('ctacargo',
                format_string('%04d%04d%012d',
                              base_contestacao_df.codent.cast(IntegerType()),
                              blah_df.numagencia_fix.cast(IntegerType()),
                              base_contestacao_df.numconta.cast(IntegerType()))) \
    .drop(base_contestacao_df.codent) \
    .drop(base_contestacao_df.numagencia) \
    .drop(blah_df.numagencia_fix) \
    .drop(base_contestacao_df.numconta)

debito_df = transactions_df.alias('transactions_df') \
    .join(
        compras_contestadas_df,
        (transactions_df.ctacargo == compras_contestadas_df.ctacargo) &
        (transactions_df.fecfac == compras_contestadas_df.dttransacao) &
        (transactions_df.impdiv == compras_contestadas_df.valorfraude),
        'left_outer') \
    .drop(compras_contestadas_df.ctacargo) \
    .withColumn('contestada', compras_contestadas_df.contalynx.isNotNull()) \
    .select(
        col('transactions_df.codent'),
        col('transactions_df.centalta'),
        col('transactions_df.cuenta'),
        transactions_df.pan,
        transactions_df.siaidcd,
        transactions_df.fecfac,
        transactions_df.impdiv,
        transactions_df.codcom,
        transactions_df.nomcomred,
        col('contestada'))

debito_autoriz_df = debito_df \
    .drop(debito_df.codcom) \
    .drop(debito_df.nomcomred) \
    .join(
        mpdt004_latest_df \
            .filter(mpdt004_latest_df.inddebcre == 2),
        debito_df.siaidcd == mpdt004_latest_df.siaidcd,
        'left_outer') \
    .select(
        col('transactions_df.codent'),
        col('transactions_df.centalta'),
        col('transactions_df.cuenta'),
        transactions_df.pan,
        transactions_df.siaidcd,
        transactions_df.fecfac,
        mpdt004_latest_df.hortrn,
        transactions_df.impdiv,
        mpdt004_latest_df.codcom,
        mpdt004_latest_df.nomcom,
        mpdt004_latest_df.idterm,
        mpdt004_latest_df.localidad,
        debito_df.contestada) \
    .persist(pyspark.StorageLevel.MEMORY_AND_DISK)
# 78 mins * 22 execs

In [6]:
debito_autoriz_df.count()

597700035

In [7]:
debito_autoriz_df.write.parquet('hdfs:/user/x181471/debito')

In [8]:
debito_autoriz_df.limit(40).toPandas()

codent centalta        cuenta               pan           siaidcd  \
0    0033     0628  660000447740  5447315417721695  0025053804499590   
1    0033     0236  660000169940  5021213417509125  0025053804499914   
2    0033     3002  660000116940  5155900464820011  0025054808040144   
3    0033     2045  660000222060  5021212284283723  0025065862908574   
4    0033     0015  660000177130  5021213357459190  0025076919569859   
5    0033     0438  660000375730  5155900374059916  0025085967491294   
6    0033     4154  120000190610  5021212684659837  0025086969460116   
7    0033     3736  660000047600  5155900379974929  0025086973558645   
8    0033     3861  120000040400  5021212646571419  0025087977588009   
9    0033     3067  660000053940  5447317726975418  0025089988236968   
10   0033     3566  660000045250  5155900379184222  0025091994306715   
11   0033     4423  660000041740  5155900374569468  0025091994308948   
12   0033     0353  660000374210  5021212296354710  0025091994309721   
13   0033     0648  660000436600  5447315425354992  0025093004340256   
14   0033     0097  660000807250  5447318327329443  0025094009049508   
15   0033     0081  660000823270  5155900380863004  0025096018246190   
16   0033     0081  660000823270  5155900380863004  0025096018246190   
17   0033     3114  660000065190  5447315549110197  0025102050351852   
18   0033     3566  660000052320  5428206086554223  0025106071890101   
19   0033     0642  660000450540  5447316333275964  0025106071892947   
20   0033     3415  120000044080  5021212720299275  0025107075906997   
21   0033     2077  660000276470  5021213081590666  0025107090988811   
22   0033     3098  660000134090  5021214062319067  0025109086638544   
23   0033     3004  660000155900  5155900303079761  0025111095996119   
24   0033     3940  660000007880  5228400078249155  0025112107052852   
25   0033     3442  660000048350  5021213128753731  0025112107053581   
26   0033     0021  660000615840  5401056007066893  0025112107053699   
27   0033     0644  660000115540  5428204430645813  0025112107054139   
28   0033     3327  660000355910  5447315590346344  0025112107056867   
29   0033     4334  660000166400  5428206034523437  0025112107057145   
30   0033     0335  660000512170  5155900373369118  0025112107057596   
31   0033     3208  660000106260  5155900384534668  0025112107057758   
32   0033     4584  660000117250  5021213406404213  0025112107057802   
33   0033     1694  660000024220  5021212216996509  0025112107058982   
34   0033     0821  120000100620  5021212637716247  0025112107061267   
35   0033     3898  660000055320  5021213155970380  0025112107063995   
36   0033     1528  660000524020  5428206032217339  0025112107068783   
37   0033     3991  660000039050  5401056008256642  0025125169268109   
38   0033     0011  660000903130  5021213477793700  0025132206960265   
39   0033     0002  120000185960  5021212543351030  0025135217576324   

       fecfac    hortrn  impdiv           codcom nomcom    idterm  \
0  2016-01-13      None   28.33             None   None      None   
1  2016-01-13      None   26.29             None   None      None   
2  2016-01-14      None  120.06             None   None      None   
3  2016-01-25      None   10.00             None   None      None   
4  2016-02-05      None   82.08             None   None      None   
5  2016-02-14  14:20:24  926.79  010179908640001         77304637   
6  2016-02-15  02:45:02    4.00  010449700700002         PV000006   
7  2016-02-15  20:14:31   23.98        010153101         PV000020   
8  2016-02-16  18:04:35   80.00  010367073230001         78058034   
9  2016-02-18  22:28:57  185.79        045208620         UN000046   
10 2016-02-20  05:19:49   30.00  010391012300001         78237305   
11 2016-02-20  05:24:45  200.00     948588813780         13780032   
12 2016-02-20  05:29:45  190.00  010607478790001         78214137   
13 2016-02-22  02:20:40    3.50  010278611450001         29571671   
14 2016-02-23  

In [9]:
debito_autoriz_df.groupBy('contestada').count().toPandas()

contestada      count
0      False  597700035

In [40]:
debito_autoriz_df.filter(debito_autoriz_df.contestada == True).select(debito_autoriz_df.siaidcd).distinct().count()

5124

In [24]:
debito_autoriz_df \
    .select(debito_autoriz_df.codcom, debito_autoriz_df.nomcom) \
    .distinct() \
    .groupBy(debito_autoriz_df.codcom) \
    .count() \
    .orderBy(col('count').desc()) \
    .limit(20) \
    .toPandas()

codcom  count
0   010731512010001  24637
1   010731508410001  23045
2   010731509570001  16939
3   010731513500001  16739
4   000001073151201  15143
5   000000001027881  13724
6   000001073150841  13605
7   000001073151350   9506
8         055155090   9192
9   000001073150957   9131
10  010158989600001   6987
11  010383314010001   5758
12  407527326000000   5665
13        058407731   5127
14  010179999930001   3642
15  000101000107586   2399
16  000000000107586   2264
17     948997993468   2038
18        052215369   1814
19  010163937060001   1794

In [16]:
debito_df.select(debito_df.codcom, debito_df.nomcomred).distinct().filter(debito_df.codcom == '010731512010001').limit(40).toPandas()

codcom               nomcomred
0   010731512010001  FRANCISCO MACHADO DE A
1   010731512010001  FARMACIA ERICKSON LTDA
2   010731512010001  DROGARIA ANAMAR DE NIL
3   010731512010001         PETZ ALPHAVILLE
4   010731512010001       PANIF PAN D AMORE
5   010731512010001        LANCHONETE UNICA
6   010731512010001   FERRAGENS E PAPELARIA
7   010731512010001        M R CONVENIENCIA
8   010731512010001           NOVA ESPORTES
9   010731512010001     M M MOREIRA REST ME
10  010731512010001          BAZAR PAP LIPE
11  010731512010001  WELLIGTON EPIFANIO ALV
12  010731512010001           PENSAO DICASA
13  010731512010001         MARLAN CALCADOS
14  010731512010001   SELIONE NOVAIS SANTOS
15  010731512010001  CAROLINE CAETANO BORGE
16  010731512010001   DOCE GULA MERCADO DOS
17  010731512010001    L FELEX MERCEARIA ME
18  010731512010001            MAYARA MODAS
19  010731512010001  CHURRASCARIA E LANC MO
20  010731512010001  KALUNGA MAT ELET HIDRA
21  010731512010001  KEETLYN VARREIRA DE MO
22  010731512010001     MERCADINHO COLHEITA
23  010731512010001  MCM CELULAR COM E SERV
24  010731512010001  VICTOR GUILHERME GONCA
25  010731512010001  DROGARIA E EPFUMARIA W
26  010731512010001        PADARIA RENASCER
27  010731512010001       RAIMUNDA FERREIRA
28  010731512010001       DROGARIA FAVORITA
29  010731512010001         REVISTARIA CARM
30  010731512010001        MERCADINHO LIONS
31  010731512010001             MCM CELULAR
32  010731512010001   NEUZA MARIA DE MORAES
33  010731512010001      ARMARINHO ALVORADA
34  010731512010001         BANCA DO ALVINO
35  010731512010001  G M SC PURCINA MERCEAR
36  010731512010001     REST REI DA CHULETA
37  010731512010001         MALHARIA PASSAR
38  010731512010001          PANIF RIBEIRAO
39  010731512010001  TARCISIO PEREIRA DA SI

In [ ]:
dataset_df = debito_df.select(
    col('siaidcd').alias('idSia'),
    col('pan').alias('cartao'),
    col('codcom').alias('idEstabelecimento'),
    col('fecfac').alias('dataHoraAutorizacao'),
    col('contestada').alias('flagContestacao'),
    col('impdiv').alias('valor'))

transactions_df = dataset_df \
    .groupBy('idSia', 'cartao', 'idEstabelecimento', 'dataHoraAutorizacao', 'flagContestacao', 'valor') \
    .agg({}) \
    .persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
# Passo 1: Extrair transações contestadas
contestacoes_df = transactions_df \
    .filter(transactions_df.flagContestacao == True) \
    .withColumnRenamed('dataHoraAutorizacao', 'dataHoraAutorizacaoContest') \
    .withColumnRenamed('valor', 'valorContest') \
    .withColumnRenamed('idSia', 'idSiaContest') \
    .drop(transactions_df.flagContestacao) \
    .cache()

# Passo 2: Transações com contestação futura
seqs_df = transactions_df \
    .join(contestacoes_df, 'cartao') \
    .filter(datediff(
        contestacoes_df.dataHoraAutorizacaoContest,
        transactions_df.dataHoraAutorizacao) > 0)

# Passo 3: Agregação de contestações futuras
contest_fut_df = seqs_df \
    .groupBy(transactions_df.idSia) \
    .agg(
        count('*').alias('contestFuturas'),
        sum('valorContest').alias('valorTotalContestFuturas'))

# Passo 4: Marcar número de contestações futuras para cada transação
enriched_transactions_df = transactions_df \
    .join(
        contest_fut_df,
        transactions_df.idSia == contest_fut_df.idSia,
        'left_outer') \
    .drop(contest_fut_df.idSia) \
    .na.fill(0, 'contestFuturas') \
    .na.fill(0, 'valorTotalContestFuturas')

# Passo 5: Agrupar buckets e cartões com estatísticas de contestações futuras
enriched_transactions_hist_df = enriched_transactions_df \
    .groupBy(
        transactions_df.idEstabelecimento,
        to_date(transactions_df.dataHoraAutorizacao).alias('date'),
        transactions_df.cartao) \
    .agg(
        count('*').alias('count'),
        first(enriched_transactions_df.contestFuturas) \
            .alias('contestFuturas'),
        first(enriched_transactions_df.valorTotalContestFuturas) \
            .alias('valorTotalContestFuturas'))

# Passo 6: Agregar estatísticas dos buckets
bucket_stats_df = enriched_transactions_hist_df \
    .groupBy(
        enriched_transactions_hist_df.idEstabelecimento,
        enriched_transactions_hist_df.date) \
    .agg(
        count('*').alias('cartoes'),
        sum('count').alias('transacoes'),
        sum(enriched_transactions_hist_df.contestFuturas) \
            .alias('transacoesContestFuturas'),
        sum(when(enriched_transactions_hist_df.contestFuturas > 0, 1) \
                .otherwise(0)) \
            .alias('cartoesContestFuturas'),
        sum(enriched_transactions_hist_df.valorTotalContestFuturas) \
            .alias('valorTotalContestFuturas'))

# Passo 7: Contagem de cartões distintos com contestações nos buckets
transactions_hist_df = contestacoes_df \
    .groupBy(
        contestacoes_df.idEstabelecimento,
        to_date(contestacoes_df.dataHoraAutorizacaoContest).alias('date'),
        contestacoes_df.cartao) \
    .agg(
        count('*').alias('count'),
        sum('valorContest').alias('valorContest'))

# Passo 8: Agregação de estatísticas presentes dos buckets
contest_bucket_stats_df = transactions_hist_df \
    .groupBy(
        transactions_hist_df.idEstabelecimento,
        transactions_hist_df.date) \
    .agg(
        count('*').alias('cartoesContest'),
        sum('count').alias('transacoesContest'),
        sum('valorContest').alias('valorTotalContest')) \
    .withColumnRenamed('idEstabelecimento', 'idEstabelecimentoContest') \
    .withColumnRenamed('date', 'dateContest')

# Passo 9: Junção de estatísticas futuras e presentes
stats_df = bucket_stats_df \
    .join(
        contest_bucket_stats_df,
        (bucket_stats_df.idEstabelecimento == contest_bucket_stats_df.idEstabelecimentoContest) &
        (bucket_stats_df.date == contest_bucket_stats_df.dateContest),
        'left_outer') \
    .drop('idEstabelecimentoContest').drop('dateContest') \
    .na.fill(0, 'cartoesContest') \
    .na.fill(0, 'transacoesContest') \
    .na.fill(0, 'valorTotalContest')

# Passo 10: Cálculo de taxas futuras/presente
results_df = stats_df \
    .withColumn('taxaCartoesContestFuturas', stats_df.cartoesContestFuturas / stats_df.cartoes) \
    .withColumn('taxaTransacoesContestFuturas', stats_df.transacoesContestFuturas / stats_df.transacoes) \
    .orderBy(col('taxaCartoesContestFuturas').desc(), stats_df.valorTotalContestFuturas.desc()) \
    .persist(pyspark.StorageLevel.MEMORY_AND_DISK)
# 29 mins / 32 execs

In [1]:
# Imprimir plano global de consulta
results_df.explain()

dat_ref_carga = 2016-09-14
InMemoryColumnarTableScan [idEstabelecimento#381,date#479,cartoes#483L,transacoes#484L,transacoesContestFuturas#485L,cartoesContestFuturas#486L,valorTotalContestFuturas#487,cartoesContest#496L,transacoesContest#497L,valorTotalContest#498,taxaCartoesContestFuturas#499,taxaTransacoesContestFuturas#500], (InMemoryRelation [idEstabelecimento#381,date#479,cartoes#483L,transacoes#484L,transacoesContestFuturas#485L,cartoesContestFuturas#486L,valorTotalContestFuturas#487,cartoesContest#496L,transacoesContest#497L,valorTotalContest#498,taxaCartoesContestFuturas#499,taxaTransacoesContestFuturas#500], true, 10000, StorageLevel(true, true, false, true, 1), (TungstenSort [taxaCartoesContestFuturas#499 DESC,valorTotalContestFuturas#487 DESC], true, 0), None)



In [2]:
'{0:,}'.format(results_df.count())

Py4JJavaError: An error occurred while calling o386.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 4 times, most recent failure: Lost task 0.3 in stage 51.0 (TID 9605, srvbdrprlbr03.santanderbr.corp): ExecutorLostFailure (executor 38 lost)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1294)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1282)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1281)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:47)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1281)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:697)
	at org.apach

In [ ]:
results_df.limit(40).toPandas()